In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_auc_score,
)
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedGroupKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler,minmax_scale,OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

pd.set_option("display.float_format", lambda x: "%.3f" % x)

import warnings

warnings.filterwarnings("ignore")






In [15]:
df=pd.read_csv("/content/Train_Data (1).csv")
df.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
0,0,tcp,netbios_dgm,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,277,15,0.000,0.000,1.000,1.000,0.050,0.060,0.000,255,15,0.060,0.060,0.000,0.000,0.000,0.000,1.000,1.000,21,1
1,0,tcp,smtp,SF,1239,400,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.000,0.000,0.000,0.000,1.000,0.000,0.000,255,116,0.450,0.040,0.000,0.000,0.110,0.000,0.020,0.000,18,0
2,0,tcp,http,SF,222,945,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,25,0.000,0.000,0.000,0.000,1.000,0.000,0.080,57,255,1.000,0.000,0.020,0.030,0.000,0.000,0.000,0.000,21,0
3,0,tcp,http,SF,235,1380,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.000,0.000,0.000,0.000,1.000,0.000,0.000,255,255,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,21,0
4,0,tcp,uucp_path,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,256,3,0.000,0.000,1.000,1.000,0.010,0.060,0.000,255,3,0.010,0.080,0.000,0.000,0.000,0.000,1.000,1.000,19,1


In [9]:
df_test=pd.read_csv("/content/Test_Data (1).csv")
df_test.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag
0,0,tcp,mtp,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,249,7,0.000,0.000,1.000,1.000,0.030,0.070,0.000,255,7,0.030,0.080,0.000,0.000,0.000,0.000,1.000,1.000,20
1,0,tcp,http,SF,199,1721,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,9,10,0.000,0.000,0.000,0.000,1.000,0.000,0.200,255,255,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,21
2,0,tcp,discard,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,6,1.000,1.000,0.000,0.000,0.380,0.380,0.000,255,14,0.050,0.090,0.000,0.000,1.000,1.000,0.000,0.000,18
3,0,tcp,telnet,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,297,10,1.000,1.000,0.000,0.000,0.030,0.050,0.000,255,2,0.010,0.090,0.000,0.000,1.000,1.000,0.000,0.000,18
4,0,tcp,exec,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,104,19,1.000,1.000,0.000,0.000,0.180,0.060,0.000,255,16,0.060,0.060,0.000,0.000,1.000,1.000,0.000,0.000,20


In [12]:
df.shape


(21712, 42)

In [13]:
df_test.shape

(21712, 42)

In [16]:
df.shape

(86845, 43)

In [17]:
data=df.copy()
data_test=df_test.copy()

In [18]:
data.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
0,0,tcp,netbios_dgm,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,277,15,0.000,0.000,1.000,1.000,0.050,0.060,0.000,255,15,0.060,0.060,0.000,0.000,0.000,0.000,1.000,1.000,21,1
1,0,tcp,smtp,SF,1239,400,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.000,0.000,0.000,0.000,1.000,0.000,0.000,255,116,0.450,0.040,0.000,0.000,0.110,0.000,0.020,0.000,18,0
2,0,tcp,http,SF,222,945,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,21,25,0.000,0.000,0.000,0.000,1.000,0.000,0.080,57,255,1.000,0.000,0.020,0.030,0.000,0.000,0.000,0.000,21,0
3,0,tcp,http,SF,235,1380,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.000,0.000,0.000,0.000,1.000,0.000,0.000,255,255,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,21,0
4,0,tcp,uucp_path,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,256,3,0.000,0.000,1.000,1.000,0.010,0.060,0.000,255,3,0.010,0.080,0.000,0.000,0.000,0.000,1.000,1.000,19,1


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86845 entries, 0 to 86844
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   duration                86845 non-null  int64  
 1   protocoltype            86845 non-null  object 
 2   service                 86845 non-null  object 
 3   flag                    86845 non-null  object 
 4   srcbytes                86845 non-null  int64  
 5   dstbytes                86845 non-null  int64  
 6   land                    86845 non-null  int64  
 7   wrongfragment           86845 non-null  int64  
 8   urgent                  86845 non-null  int64  
 9   hot                     86845 non-null  int64  
 10  numfailedlogins         86845 non-null  int64  
 11  loggedin                86845 non-null  int64  
 12  numcompromised          86845 non-null  int64  
 13  rootshell               86845 non-null  int64  
 14  suattempted             86845 non-null

In [20]:
data.duplicated().sum()

0

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86845 entries, 0 to 86844
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   duration                86845 non-null  int64  
 1   protocoltype            86845 non-null  object 
 2   service                 86845 non-null  object 
 3   flag                    86845 non-null  object 
 4   srcbytes                86845 non-null  int64  
 5   dstbytes                86845 non-null  int64  
 6   land                    86845 non-null  int64  
 7   wrongfragment           86845 non-null  int64  
 8   urgent                  86845 non-null  int64  
 9   hot                     86845 non-null  int64  
 10  numfailedlogins         86845 non-null  int64  
 11  loggedin                86845 non-null  int64  
 12  numcompromised          86845 non-null  int64  
 13  rootshell               86845 non-null  int64  
 14  suattempted             86845 non-null

In [45]:
df.isnull().sum()
df=df.dropna()

In [46]:
df.describe()

,duration,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
count,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000,86845.000
mean,104.167,8455.707,2732.095,0.000,0.000,0.000,0.145,0.001,0.441,0.321,0.001,0.001,0.356,0.014,0.000,0.005,0.000,0.000,0.008,79.585,21.309,0.325,0.323,0.091,0.091,0.639,0.045,0.079,186.012,123.338,0.526,0.050,0.076,0.016,0.325,0.319,0.092,0.091,20.061,0.380
std,1038.274,358214.056,55430.412,0.007,0.000,0.008,1.838,0.037,0.497,27.993,0.037,0.048,28.415,0.529,0.020,0.105,0.000,0.003,0.089,98.068,48.212,0.465,0.464,0.286,0.286,0.445,0.122,0.222,95.947,112.704,0.447,0.103,0.208,0.055,0.464,0.464,0.283,0.281,1.421,0.485
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
25%,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,2.000,0.000,0.000,0.000,0.000,0.090,0.000,0.000,93.000,12.000,0.050,0.000,0.000,0.000,0.000,0.000,0.000,0.000,19.000,0.000
50%,0.000,46.000,44.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,16.000,8.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,255.000,77.000,0.550,0.030,0.000,0.000,0.000,0.000,0.000,0.000,21.000,0.000
75%,0.000,272.000,768.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,143.000,17.000,1.000,1.000,0.000,0.000,1.000,0.060,0.000,255.000,255.000,1.000,0.070,0.020,0.010,1.000,1.000,0.000,0.000,21.000,1.000
max,40504.000,89581520.000,7028652.000,1.000,0.000,2.000,77.000,4.000,1.000,7479.000,1.000,2.000,7468.000,43.000,2.000,9.000,0.000,1.000,1.000,511.000,511.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,255.000,255.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,21.000,1.000


In [48]:
df.protocoltype.unique()

array(['tcp', 'udp', 'icmp'], dtype=object)

In [49]:
df.service.unique()

array(['netbios_dgm', 'smtp', 'http', 'uucp_path', 'private', 'csnet_ns',
       'hostnames', 'IRC', 'courier', 'ssh', 'Z39_50', 'other',
       'ftp_data', 'efs', 'exec', 'telnet', 'time', 'domain_u', 'auth',
       'iso_tsap', 'http_443', 'nnsp', 'gopher', 'vmnet', 'netstat',
       'whois', 'netbios_ns', 'nntp', 'daytime', 'mtp', 'discard', 'echo',
       'netbios_ssn', 'link', 'pop_3', 'ctf', 'ldap', 'uucp', 'ecr_i',
       'finger', 'systat', 'klogin', 'ftp', 'sunrpc', 'sql_net', 'name',
       'kshell', 'domain', 'imap4', 'urp_i', 'login', 'supdup', 'eco_i',
       'rje', 'ntp_u', 'bgp', 'pop_2', 'printer', 'X11', 'tim_i', 'shell',
       'remote_job', 'urh_i', 'tftp_u', 'red_i'], dtype=object)

In [50]:
df.flag.unique()

array(['REJ', 'SF', 'S0', 'RSTR', 'RSTO', 'S1', 'S3', 'S2', 'SH', 'OTH'],
      dtype=object)

In [51]:
num_cols = df.select_dtypes(exclude=["object"]).columns
cat_cols = df.select_dtypes(include=["object"]).columns

In [52]:
cat_cols

Index(['protocoltype', 'service', 'flag'], dtype='object')

In [53]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])


In [54]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [55]:
df.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag,attack
0,0.000,1,33,1,-0.169,-0.057,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.851,0.467,0.000,0.000,1.000,1.000,-1.044,1.000,0.000,0.000,-0.255,-0.516,0.429,0.000,0.000,0.000,0.000,1.000,1.000,0.000,1.000
1,0.000,1,49,8,4.386,0.464,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.106,-0.467,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.160,-0.105,0.143,0.000,0.000,0.110,0.000,0.020,0.000,-1.500,0.000
2,0.000,1,22,8,0.647,1.173,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.035,1.133,0.000,0.000,0.000,0.000,0.000,0.000,0.080,-1.222,0.733,0.474,-0.429,1.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,1,22,8,0.695,1.740,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.085,-0.267,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.733,0.474,-0.429,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,1,62,1,-0.169,-0.057,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.702,-0.333,0.000,0.000,1.000,1.000,-1.088,1.000,0.000,0.000,-0.305,-0.568,0.714,0.000,0.000,0.000,0.000,1.000,1.000,-1.000,1.000


In [56]:
x=df.drop("attack",axis=1)
y=df["attack"]
x.isnull().sum()

duration                  0
protocoltype              0
service                   0
flag                      0
srcbytes                  0
dstbytes                  0
land                      0
wrongfragment             0
urgent                    0
hot                       0
numfailedlogins           0
loggedin                  0
numcompromised            0
rootshell                 0
suattempted               0
numroot                   0
numfilecreations          0
numshells                 0
numaccessfiles            0
numoutboundcmds           0
ishostlogin               0
isguestlogin              0
count                     0
srvcount                  0
serrorrate                0
srvserrorrate             0
rerrorrate                0
srvrerrorrate             0
samesrvrate               0
diffsrvrate               0
srvdiffhostrate           0
dsthostcount              0
dsthostsrvcount           0
dsthostsamesrvrate        0
dsthostdiffsrvrate        0
dsthostsamesrcportra

In [58]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn import metrics

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

from sklearn.impute import SimpleImputer

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import RandomizedSearchCV

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

pd.set_option("display.float_format", lambda x: "%.3f" % x)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

pd.set_option("display.float_format", lambda x: "%.3f" % x)

import warnings

warnings.filterwarnings("ignore")


In [59]:
models = []
models.append(
    ("Logistic Regression", LogisticRegression(random_state=42))
)
models.append(("decisiontree", DecisionTreeClassifier(random_state=42)))
models.append(("Random forest", RandomForestClassifier(random_state=42)))
models.append(("Bagging", BaggingClassifier(random_state=42)))
models.append(("Adaboost", AdaBoostClassifier(random_state=42)))
models.append(("GBM", GradientBoostingClassifier(random_state=42)))
models.append(("Xgboost classifier", XGBClassifier(random_state=42, eval_metric="logloss")))
results = []
names = []
score = []
print("\n" "Cross-Validation Performance:" "\n")
scorer = "f1"
for name, model in models:
    kfold = StratifiedKFold(
        n_splits=5, shuffle=True, random_state=42)
    cv_result = cross_val_score(estimator=model, X=x, y=y, scoring=scorer, cv=kfold )
    results.append(cv_result)
    names.append(name)
    print("{}: {}".format(name, cv_result.mean()))



Cross-Validation Performance:

Logistic Regression: 0.9998333115108522
decisiontree: 0.9997122347672315
Random forest: 1.0
Bagging: 0.9997424483148478
Adaboost: 0.9999696946738389
GBM: 0.9998636237285897
Xgboost classifier: 0.9999090954973291


In [60]:
for name,model in models:
    model.fit(x,y)
    scores = model.score(x,y)
    print("{}:{}".format(name,scores))

Logistic Regression:1.0
decisiontree:1.0
Random forest:1.0
Bagging:0.9999654556969313
Adaboost:1.0
GBM:1.0
Xgboost classifier:1.0


In [61]:
num_cols = df_test.select_dtypes(exclude=["object"]).columns
cat_cols = df_test.select_dtypes(include=["object"]).columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cat_cols:
    df_test[col] = le.fit_transform(df_test[col])
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
df_test[num_cols] = scaler.fit_transform(df_test[num_cols])

In [62]:
x_test=df_test


In [64]:
x_test.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,rootshell,suattempted,numroot,numfilecreations,numshells,numaccessfiles,numoutboundcmds,ishostlogin,isguestlogin,count,srvcount,serrorrate,srvserrorrate,rerrorrate,srvrerrorrate,samesrvrate,diffsrvrate,srvdiffhostrate,dsthostcount,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag
0,0.000,1,31,1,-0.168,-0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.652,-0.067,0.000,0.000,1.000,1.000,-1.066,1.167,0.000,0.000,-0.296,-0.558,0.857,0.000,0.000,0.000,0.000,1.000,1.000,-0.500
1,0.000,1,22,8,0.560,2.187,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.050,0.133,0.000,0.000,0.000,0.000,0.000,0.000,0.200,0.000,0.724,0.463,-0.286,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,1,9,4,-0.168,-0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.133,1.000,1.000,0.000,0.000,-0.681,6.333,0.000,0.000,-0.267,-0.537,1.000,0.000,0.000,1.000,1.000,0.000,0.000,-1.500
3,0.000,1,55,4,-0.168,-0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.993,0.133,1.000,1.000,0.000,0.000,-1.066,0.833,0.000,0.000,-0.317,-0.579,1.000,0.000,0.000,1.000,1.000,0.000,0.000,-1.500
4,0.000,1,16,4,-0.168,-0.059,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.624,0.733,1.000,1.000,0.000,0.000,-0.901,1.000,0.000,0.000,-0.259,-0.526,0.571,0.000,0.000,1.000,1.000,0.000,0.000,-0.500


In [65]:
rf=RandomForestClassifier()
rf.fit(x,y)
y_pred=rf.predict(x_test)

In [66]:
y_pred


array([1., 0., 1., ..., 1., 0., 1.])

In [68]:
def swap_attack_normal(value):
    if value == 0:
        return 1
    else:
        return 0
submission = pd.DataFrame({'attack':y_pred})
submission['attack'] = submission['attack'].apply(swap_attack_normal)
submission.to_csv('submission_aiplanett.csv', index=False)

In [69]:
submission.head()

,attack
0,0
1,1
2,0
3,0
4,0


In [70]:
submission.shape

(21712, 1)